In [1]:
import bs4 as bs
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import pandas as pd
import re

In [2]:

myUrl= 'https://www.humblebundle.com/pc-lovers-software-bundle'

uClient = uReq(myUrl)

page_html = uClient.read()

uClient.close()

In [3]:
page_soup = soup(page_html,"html.parser")

In [6]:
page_soup.title

<title>Humble Software Bundle for PC Lovers (pay what you want and help charity)</title>

## Traversing Elements 

In [9]:
game_containers = page_soup.find_all('div',{'class':'main-content-row dd-game-row js-nav-row'})


In [16]:
game_cat_free = game_containers[0]
game_cat_free

<div class="main-content-row dd-game-row js-nav-row">
<div class="u-constrain-width">
<div class="dd-header">
<h2 class="dd-header-headline s-fixed">
    Pay what you want!
  </h2>
<h3 class="dd-header-subheader s-fixed">
</h3>
</div>
<div class="dd-image-box-list">
<div class="dd-image-box game-boxes hoverable desktop" data-slideout="systemmechanic165">
<div class="dd-image-box-figure u-lazy-load">
<div class="dd-image-box-badge-holder u-hide-onerror">
</div>
<div class="dd-image-holder">
<img class="dd-image-box-figure-img" data-retina-src="https://humblebundle.imgix.net/misc/files/hashed/4e6dbda9ab731e948eb8256c384d15d7b1b6e61a.png?auto=format&amp;dpr=2&amp;fit=clip&amp;h=240&amp;lossless=1&amp;w=180&amp;ixlib=python-0.2.0&amp;s=b9094461768d5342c28830c9c2000966" data-src="https://humblebundle.imgix.net/misc/files/hashed/4e6dbda9ab731e948eb8256c384d15d7b1b6e61a.png?auto=format&amp;fit=clip&amp;h=240&amp;lossless=1&amp;w=180&amp;ixlib=python-0.2.0&amp;s=ff30618c5df363ed367717ed6e8e6cf

In [33]:
game_category= game_cat_free.h2.text.strip()
game_category

'Pay what you want!'

In [96]:
game_titlez=[]
game_msrpz=[]
game_subtitlez=[]

In [101]:
game_titles = game_cat_free.findAll('div',{'class':'dd-image-box-caption'})
for title in game_titles:
    game_titlez.append(title.text.strip())
game_subtitles = game_cat_free.findAll('div',{'class':"dd-callout subtitle"})
for subs in game_subtitles:
    game_subtitlez.append(subs.text.strip().split("MSRP")[0])
game_msrps = game_cat_free.findAll('p',{'class':"callout-msrp"})
for msrp in game_msrps: 
    msrp = re.findall('\d+.\d+',msrp.text.split(" ")[1])
    if len(msrp)>0:
        game_msrpz.append(msrp[0])
    else:
        game_msrpz.append("")


In [104]:
print(len(game_titlez))
print(len(game_msrpz))
print(len(game_subtitlez))

6
6
6


In [127]:
game_dictionary ={}

In [110]:
len(game_containers)

3

In [128]:
i=0
while(i<len(game_containers)):
    gameContainerName = game_containers[i]
    game_category= gameContainerName.h2.text.strip()
    games_in_container =[]
    game_titlez=[]
    game_msrpz=[]
    game_subtitlez=[]
    game_titles = gameContainerName.findAll('div',{'class':'dd-image-box-caption'})
    for title in game_titles:
        game_titlez.append(title.text.strip())
    game_subtitles = gameContainerName.findAll('div',{'class':"dd-callout subtitle"})
    for subs in game_subtitles:
        game_subtitlez.append(subs.text.strip().split("MSRP")[0])
    game_msrps = gameContainerName.findAll('p',{'class':"callout-msrp"})
    for msrp in game_msrps: 
        msrp = re.findall('\d+.\d+',msrp.text.split(" ")[1])
        if len(msrp)>0:
            game_msrpz.append(msrp[0])
        else:
            game_msrpz.append("")
    
    gameIndex=0
    while(gameIndex<len(game_titlez)):
        game = [game_titlez[gameIndex],game_subtitlez[gameIndex],game_msrpz[gameIndex]]
        games_in_container.append(game)
#         games_in_container[game_titlez[gameIndex]].update({game})
        gameIndex+=1
    
    game_dictionary[game_category] = games_in_container
    
    i+=1



    

In [136]:
game_dictionary

{'Pay $12 or more to also unlock!': [['PCMark 8 Advanced Edition',
   '',
   '49.99'],
  ['VRMark Advanced Edition', '', '19.99'],
  ['Dashlane Premium', '1 year, new users only\n', '39.99']],
 'Pay more than the average of $6.22 to also unlock!': [['DAEMON Tools Pro 8 Lifetime',
   'Lifetime licenses for up to 3 PCs\n',
   '59.99'],
  ['Backblaze Online Backup',
   '6 months of unlimited backup, for new users only\n',
   '30.00'],
  ['3DMark Advanced Edition', '', '29.99']],
 'Pay what you want!': [['System Mechanic 16.5',
   '1 year license for new customers only',
   '49.95'],
  ['AirParrot 2 (two-pack)', 'Lifetime licenses, redeemable twice', '12.99'],
  ['EaseUS Partition Master Pro 11.9', '', '39.95']]}

In [137]:
df = pd.DataFrame.from_dict(game_dictionary, orient="index")
df = df.T
df

,Pay what you want!,Pay more than the average of $6.22 to also unlock!,Pay $12 or more to also unlock!
0,"[System Mechanic 16.5, 1 year license for new ...","[DAEMON Tools Pro 8 Lifetime, Lifetime license...","[PCMark 8 Advanced Edition, , 49.99]"
1,"[AirParrot 2 (two-pack), Lifetime licenses, re...","[Backblaze Online Backup, 6 months of unlimite...","[VRMark Advanced Edition, , 19.99]"
2,"[EaseUS Partition Master Pro 11.9, , 39.95]","[3DMark Advanced Edition, , 29.99]","[Dashlane Premium, 1 year, new users only\n, 3..."


In [139]:
df.to_csv("Humble_Bundle_PC_Softwares.csv")